In [186]:
import pandas as pd
import numpy as np
import re
import string

In [187]:
from nltk.stem.snowball import SnowballStemmer

In [188]:
stemmer = SnowballStemmer("english")

In [189]:
pd.set_option('display.max_colwidth', None)

In [190]:
train = pd.read_csv("train.csv",encoding = "ISO-8859-1")
train.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [191]:
test = pd.read_csv("test.csv",encoding = "ISO-8859-1")
test.head(5)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, stay safe everyone."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all"
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


## Cuento los links de cada tweet: (tengo que hacerlo antes de limpiarlos porque en la limpieza se pierden)

In [192]:
train['cantidad_links'] = train['text'].str.count('http')

In [193]:
test['cantidad_links'] = test['text'].str.count('http')

In [194]:
train.nlargest(30,'cantidad_links').head()

,id,keyword,location,text,target,cantidad_links
40,59,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http://t.co/3Tj8ZjiN21 http://t.co/YDUiXEfIpE http://t.co/LxTjc87KLS #nsfw,0,4
48,68,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http://t.co/3Tj8ZjiN21 http://t.co/YDUiXEfIpE http://t.co/LxTjc87KLS #nsfw,0,4
7082,10143,upheaval,Attock,Ancient Mayan Tablet found via http://t.co/LmUMzkLtln http://t.co/yebxxAryBF http://t.co/SRRUqfffr6 http://t.co/CadzxAgMSI,0,4
1060,1531,bomb,NaN,beforeitsnews : Global Derivatives: $1.5 Quadrillion Time Bomb http://t.co/GhmmUj7GbE (vÂÃ_ http://t.co/u9LvvLzhYe) http://t.co/LyJ57pq3yX,0,3
1373,1980,bush%20fires,Trinidad and Tobago,Drought fuels bush fires in Jamaica - http://t.co/ZDtDqQbAHC http://t.co/PsQCNsVfgP - @JamaicaObserver @cnewslive RE https://t.co/6ZGef8J8Bm,1,3


## Signos de pregunta

In [195]:
test['cantidad_signos_pregunta'] = test['text'].str.count('\?')

In [196]:
train['cantidad_signos_pregunta'] = train['text'].str.count('\?')

## Limpieza de los textos

In [197]:
train_limpio = train.copy()

In [198]:
test_limpio = test.copy()

In [199]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()  #quito_mayusculas
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  #quito caracteres inusuales
    text = re.sub('â' , '', text)  #quito caracteres inusuales
    text = re.sub('ª' , '', text)  #quito caracteres inusuales
    text = re.sub('ã' , '', text)  #quito caracteres inusuales
    text = re.sub('¼' , '', text)  #quito caracteres inusuales
    text = re.sub('\w*\d\w*', '', text)  # quito numeros
    return text

In [200]:
train_limpio['text'] = train_limpio['text'].apply(lambda x: clean_text(x))
train_limpio.head()

,id,keyword,location,text,target,cantidad_links,cantidad_signos_pregunta
0,1,NaN,NaN,our deeds are the reason of this earthquake may allah forgive us all,1,0,0
1,4,NaN,NaN,forest fire near la ronge sask canada,1,0,0
2,5,NaN,NaN,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,0
3,6,NaN,NaN,people receive wildfires evacuation orders in california,1,0,0
4,7,NaN,NaN,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0,0


In [201]:
test_limpio['text'] = test_limpio['text'].apply(lambda x: clean_text(x))

## Analizar las repeticiones de un tweet

In [202]:
train_limpio['repeticiones'] = train_limpio.groupby('text')['id'].transform('count')
train_limpio.head()

,id,keyword,location,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones
0,1,NaN,NaN,our deeds are the reason of this earthquake may allah forgive us all,1,0,0,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1,0,0,1
2,5,NaN,NaN,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,0,1
3,6,NaN,NaN,people receive wildfires evacuation orders in california,1,0,0,1
4,7,NaN,NaN,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0,0,1


In [203]:
train_limpio.nlargest(20,'repeticiones').head()

,id,keyword,location,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones
5876,8393,sandstorm,USA,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,22
5877,8394,sandstorm,NaN,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,22
5879,8397,sandstorm,NaN,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,22
5882,8401,sandstorm,hkXfYMhEx,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,22
5883,8402,sandstorm,US,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,22


In [204]:
test_limpio['repeticiones'] = test_limpio.groupby('text')['id'].transform('count')

## Voy a dejar los keywords con stemming

### Con esto busco que los keywords me ayuden a decidir el target pero solo lo hagan si hay "suficientes" como para hacerlo.

### Por ejemplo, queremos poder decidir el target por el keyword si hay 6 

In [205]:
rowsWithNonNullKeyword = train_limpio.loc[train_limpio.keyword.notnull()]
rowsWithNonNullKeyword[:4]

,id,keyword,location,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones
31,48,ablaze,Birmingham,bbcmtd wholesale markets ablaze,1,1,0,1
32,49,ablaze,Est. September 2012 - Bristol,we always try to bring the heavy metal rt,0,1,0,1
33,50,ablaze,AFRICA,africanbaze breaking newsnigeria flag set ablaze in aba,1,1,0,1
34,52,ablaze,"Philadelphia, PA",crying out for more set me ablaze,0,0,0,1


In [206]:
keywordsCount = rowsWithNonNullKeyword.groupby("keyword").agg({"id": "count"}).reset_index()
keywordsCount.columns = ["keyword", "count"]
keywordsCount.head(2)

,keyword,count
0,ablaze,36
1,accident,35


In [207]:
train_limpio['keyword'] = train_limpio['keyword'].replace({ np.nan:'null'})
train_limpio["keyword"] = train_limpio['keyword'].apply(lambda x: ' '.join([stemmer.stem(y) for y in x.split(' ')]))

In [208]:
test_limpio['keyword'] = test_limpio['keyword'].replace({ np.nan:'null'})
test_limpio["keyword"] = test_limpio['keyword'].apply(lambda x: ' '.join([stemmer.stem(y) for y in x.split(' ')]))

## Sabemos que hay locations repetidas pero con distinto nombre, veo si puedo agrupar

In [209]:
rowsWithNonNullLocation = train_limpio.loc[train_limpio.location.notnull()].copy()
rowsWithNonNullLocation

,id,keyword,location,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones
31,48,ablaz,Birmingham,bbcmtd wholesale markets ablaze,1,1,0,1
32,49,ablaz,Est. September 2012 - Bristol,we always try to bring the heavy metal rt,0,1,0,1
33,50,ablaz,AFRICA,africanbaze breaking newsnigeria flag set ablaze in aba,1,1,0,1
34,52,ablaz,"Philadelphia, PA",crying out for more set me ablaze,0,0,0,1
35,53,ablaz,"London, UK",on plus side look at the sky last night it was ablaze,0,1,0,1
...,...,...,...,...,...,...,...,...
7575,10826,wreck,TN,on the bright side i wrecked,0,1,0,1
7577,10829,wreck,#NewcastleuponTyne #UK,hes gone you can relax i thought the wife who wrecked her cake was a goner mind lol whoops,0,0,0,1
7579,10831,wreck,"Vancouver, Canada",three days off from work and theyve pretty much all been wrecked hahaha shoutout to my family for that one,0,0,0,1
7580,10832,wreck,London,fx forex trading cramer igers words that wrecked disneys stock,0,1,0,1


In [210]:
locationCount = rowsWithNonNullLocation.groupby("location").agg({"id":"count"}).reset_index()
locationCount.columns = ["location", "count"]
locationCount.loc[locationCount["count"] > 4]

,location,count
27,Road to the Billionaires Club,6
114,304,9
175,??????,6
324,"Asheville, NC",5
339,Atlanta,6
...,...,...
2866,WorldWide,5
2868,Worldwide,19
3054,in the Word of God,5
3222,ss,10


### Casos como New York, new york, ny y nyc nos pueden molestar, veo si hay manera de arreglarlo

In [211]:
#list(locationCount.loc[locationCount["count"] > 4]["location"])

#Asheville, NC | Atlanta | Atlanta, GA | Austin, TX | Australia | Boston, MA | Brasil | Brooklyn, NY | Calgary, AB | Calgary, Alberta
#California | California, USA | California, United States | Canada | Charlotte, NC | Chicago | Chicago, IL | Cleveland, OH
#Colorado | Coventry | Dallas, TX | Denver, CO | Denver, Colorado | Earth | Everywhere | Florida | Florida, USA
#Global | Houston, TX | India | Indiana | Indonesia | Ireland | Kenya | Lagos | Lagos, Nigeria | Leeds, England
#London | London, England | London, UK | Los Angeles | Los Angeles, CA | Manchester | Massachusetts
#Melbourne, Australia | Memphis, TN | Morioh, Japan | Mumbai | NY | NYC | Nashville, TN | New Hampshire
#New Jersey | New York | New York City | New York, NY | New York, USA | Newcastle | Nigeria | North Carolina
#Oklahoma City, OK | Orland, FL | Pakistan | Paterson, New Jersey | Pedophile hunting ground | Pennsylvania
#Pennsylvania, USA | Philippines | Planet Earth | Portland, OR | Puerto Rico | Sacramento, CA | San Diego, CA
# San Francisco | San Francisco, CA | San Jose, CA | Scotland | Seattle | Seattle, WA | Singapore | South Africa
# Southern California | Sydney | Tampa, FL | Tennessee | Texas | Texas, USA | The Netherlands | Toronto | UK
# US | USA | United Kingdom | United States | Vancouver, BC | Washington, D.C. | Washington, DC | Winsconsin
# World | WorldWide | Worldwide | in the Word of God | ss | worldwide


In [212]:
realPlaces = {"NC": "NC", "Atlanta": "Atlanta", "TX": "TX", "Texas": "TX", "Australia": "Australia", "Boston": "MA", "MA": "MA", "Brasil": "Brasil", "Brooklyn": "NY", "Calgary": "AB", "AB": "AB", "California": "California", "Canada": "Canada", "Chicago": "Chicago", "Cleveland": "Cleveland", "Colorado": "Colorado", "Coventry": "Coventry", "Dallas": "TX", "Denver": "CO", "CO": "CO", "Colorado": "CO", "Florida": "Florida", "Houston": "TX", "India": "India", "Indiana": "Indiana", "Indonesia": "Indonesia", "Ireland": "Ireland", "Kenya": "Kenya", "Lagos": "Lagos", "Leeds": "UK", "London": "UK", "UK": "UK", "Los Angeles": "Los Angeles", "Manchester": "Manchester", "Massachusetts": "Massachusetts", "TN": "TN", "Japan": "Japan", "Mumbai": "Mumbai", "NY": "NY", "NYC": "NYC", "New Hampshire": "New Hampshire", "New Jersey": "New Jersey", "New York": "NY", "New York City": "NYC", "USA": "USA", "Newcastle": "Newcastle", "Nigeria": "Nigeria", "North Carolina": "NC", "Oklahoma City": "Oklahoma City", "Orland": "Orland", "Pakistan": "Pakistan", "Pennsylvania": "Pennsylvania", "Philippines": "Philippines", "Portland": "Portland", "Puerto Rico": "Puerto Rico", "Sacramento": "Sacramento", "San Diego": "San Diego", "San Francisco": "San Francisco", "San Jose": "San Jose", "Scotland": "Scotland", "Seattle": "Seattle", "Singapore": "Singapore", "South Africa": "South Africa", "Southern California": "Southern California", "Sydney": "Sydney", "Tampa": "Tampa", "Tennesse": "Tennesse", "Texas": "TX", "The Netherlands": "The Netherlands", "Toronto": "Toronto", "UK": "UK", "US": "USA", "USA": "USA", "United Kingdom": "UK", "United States": "USA", "Vancouver": "Vancouver", "Washington": "Washington", "Winsconsin": "Winsconsin"}                                                                                                                                 
realPlacesWithSpaces = ["Los Angeles", "New Hampshire", "New Jersey", "New York", "North Carolina", "Oklahoma City", "Puerto Rico", "San Diego", "San Francisco", "San Jose", "South Africa", "Southern California", "The Netherlands", "United Kingdom", "United States"]
notRealPlaces = ["Earth", "Everywhere", "Global", "Pedophile hunting ground", "Planet Earth", "World", "WorldWide", "Worldwide", "in the Word of God", "ss", "worldwide"]

In [213]:
def getLocationGroup(location):
    if location in notRealPlaces:
        return location
    else:
        for place in realPlacesWithSpaces:
            if(place in location):
                return realPlaces[place]
        locationString = location.replace(",", "")
        for word in locationString.split(" "):
            if word in realPlaces:
                return realPlaces[word]
        return location
    
def isRealPlace(location):
    return getLocationGroup(location) in realPlaces

train_limpio["location"] = train_limpio["location"].replace({ np.nan:'null'})
train_limpio["location"] = train_limpio["location"].map(lambda x: getLocationGroup(x))
train_limpio["isRealPlace"] = train_limpio["location"].map(lambda x: isRealPlace(x))

test_limpio["location"] = test_limpio["location"].replace({ np.nan:'null'})
test_limpio["location"] = test_limpio["location"].map(lambda x: getLocationGroup(x))
test_limpio["isRealPlace"] = test_limpio["location"].map(lambda x: isRealPlace(x))

In [214]:
#Remuevo informacion
##Remuevo el ID

In [282]:
train_limpio = train_limpio.drop(columns=['id'])
train_limpio.head(2)

,keyword,location,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones,isRealPlace,longitud_tweet,letras_seguidas,...,into effect,insultos_por_tweet,expresiones_por_tweet,bombing,suicide,accident,disaster,police,died,years
0,null,null,our deed are the reason of this earthquak may allah forgiv us all,1,0,0,1,False,70,2,...,0,0,0,0,0,0,0,0,0,0
1,null,null,forest fire near la rong sask canada,1,0,0,1,False,40,1,...,0,0,0,0,0,0,0,0,0,0


## Remuevo los datos repetidos 
### No remuevo los repetivos del test obviamente

In [216]:
train_limpio.shape

(7613, 8)

In [217]:
train_limpio = train_limpio.drop_duplicates()
train_limpio.shape

(7389, 8)

In [218]:
train_limpio.head()

,keyword,location,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones,isRealPlace
0,null,null,our deeds are the reason of this earthquake may allah forgive us all,1,0,0,1,False
1,null,null,forest fire near la ronge sask canada,1,0,0,1,False
2,null,null,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,0,1,False
3,null,null,people receive wildfires evacuation orders in california,1,0,0,1,False
4,null,null,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0,0,1,False


# Longitud de los tweets

In [219]:
import math

def roundup(x):
    return int(math.ceil(x / 10.0)) * 10

In [220]:
train_limpio['longitud_tweet'] = train_limpio['text'].apply(lambda x: roundup(len(x)))

In [221]:
train_limpio.head(2)

,keyword,location,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones,isRealPlace,longitud_tweet
0,null,null,our deeds are the reason of this earthquake may allah forgive us all,1,0,0,1,False,70
1,null,null,forest fire near la ronge sask canada,1,0,0,1,False,40


In [222]:
test_limpio['longitud_tweet'] = test_limpio['text'].apply(lambda x: roundup(len(x)))

## Contar la cantidad de veces seguidas que se repite una letra, si alguna se repite mas de 3 veces, es probable que haya sido intencional, lo cual le quita credibildad

In [223]:
def letras_repetidas(text):
    max_repeteated_letters = 0
    current_repeated_letters = 1
    for word in text:
        if(current_repeated_letters>max_repeteated_letters):
            max_repeteated_letters=current_repeated_letters
        current_repeated_letters=1
        for i in range (len(word)-1):
            if(word[i]==word[i+1]):
                current_repeated_letters=current_repeated_letters+1
            else:
                if(current_repeated_letters>max_repeteated_letters):
                    max_repeteated_letters=current_repeated_letters
                current_repeated_letters=1
    return (max_repeteated_letters)

In [224]:
def remover_palabras_con_letras_repetidas(text):
    max_repeated_letters = 0
    current_repeated_letters = 1
    new_text = []
    for word in text:
        current_repeated_letters=1
        max_repeated_letters = 1
        for i in range (len(word)-1):
            if(word[i]==word[i+1]):
                current_repeated_letters=current_repeated_letters+1
            else:
                if(current_repeated_letters>max_repeated_letters):
                    max_repeated_letters=current_repeated_letters
                current_repeated_letters=1
        if(current_repeated_letters<=2 and max_repeated_letters<=2):
            new_text.append(word)  #remuevo las palabras que tengan 3 o mas letras seguidas repetidas
    return ( ' '.join(new_text))

In [225]:
train_limpio['letras_seguidas'] = train_limpio['text'].apply(lambda x: letras_repetidas(x.split(' ')))

test_limpio['letras_seguidas'] = test_limpio['text'].apply(lambda x: letras_repetidas(x.split(' ')))

In [226]:
letras_seguidas_train = train_limpio['letras_seguidas']
letras_seguidas_test = test_limpio['letras_seguidas']

In [227]:
#Remuevo esas palabras, x ahora las dejo
train_limpio['text'] = train_limpio['text'].apply(lambda x: remover_palabras_con_letras_repetidas(x.split(' ')))
test_limpio['text'] = test_limpio['text'].apply(lambda x: remover_palabras_con_letras_repetidas(x.split(' ')))
train_limpio.head(2)

,keyword,location,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones,isRealPlace,longitud_tweet,letras_seguidas
0,null,null,our deeds are the reason of this earthquake may allah forgive us all,1,0,0,1,False,70,2
1,null,null,forest fire near la ronge sask canada,1,0,0,1,False,40,1


### Parece dar buenos resultados pero obviamente no reemplaza un programa que me detecte cualquier error ortografico

# Correccion de typos

### Lo deje comentado por ser demasiado lento

In [228]:
#Prueba basica
#pruebita = train_limpio.iloc[1:150].copy()

def spell_correcter(text):
    word_list = text.split(' ')
    return [str(TextBlob(word).correct()) for word in word_list]

#from textblob import TextBlob
#pruebita['text'] = pruebita['text'].apply(lambda x: spell_correcter(x))
#pruebita.text = pruebita.text.apply(lambda txt: ''.join(TextBlob(txt).correct()))

#pruebita['text_2'] = pruebita['text'].apply(lambda x: ' '.join(x))
#pruebita.head()

,keyword,location,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones,isRealPlace,longitud_tweet,letras_seguidas,text_2
1,null,null,"[forest, fire, near, la, range, ask, canada]",1,0,0,1,False,40,1,forest fire near la range ask canada
2,null,null,"[all, residents, asked, to, shelter, in, place, are, being, notified, by, officers, no, other, evacuation, or, shelter, in, place, orders, are, expected]",1,0,0,1,False,130,2,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected
3,null,null,"[, people, receive, wildfires, evacuation, orders, in, california, ]",1,0,0,1,False,60,1,people receive wildfires evacuation orders in california
4,null,null,"[just, got, sent, this, photo, from, ruby, alaska, as, smoke, from, wildfires, hours, into, a, school, ]",1,0,0,1,False,90,2,just got sent this photo from ruby alaska as smoke from wildfires hours into a school
5,null,null,"[rockyfire, update, , california, why, , closed, in, both, directions, due, to, lake, county, fire, , fire, wildfires]",1,0,0,1,False,110,1,rockyfire update california why closed in both directions due to lake county fire fire wildfires


In [229]:
#Confirmé que funciona, lo aplico a todas

In [230]:
#train_limpio['text'] = train_limpio['text'].apply(lambda x: spell_correcter(x))

#train_limpio['text'] = train_limpio['text'].apply(lambda x: ' '.join(x))

#test_limpio['text'] = test_limpio['text'].apply(lambda x: spell_correcter(x))

#test_limpio['text'] = test_limpio['text'].apply(lambda x: ' '.join(x))

In [231]:
#train_limpio.to_csv('train_con_textos_corregidos.csv', index=False)

#test_limpio.to_csv('test_con_textos_corregidos.csv', index=False)

In [232]:
#train_texto_corregido = pd.read_csv("train_con_textos_corregidos.csv",encoding = "ISO-8859-1")
#test_texto_corregido = pd.read_csv("test_con_textos_corregidos.csv",encoding = "ISO-8859-1")

In [233]:
#train_texto_corregido = train_texto_corregido.text.fillna(0)

In [234]:
#test_texto_corregido = test_texto_corregido.text.fillna(0)

In [235]:
#train_limpio['texto_corregido'] = train_texto_corregido

In [236]:
#test_limpio['texto_corregido'] = test_texto_corregido

In [237]:
#train_limpio[train_limpio['texto_corregido']!=train_limpio['text']].head()

In [238]:
#Corrige typos que no deberia, mejor no lo uso

# Analisis de sentimiento y de objetividad

In [239]:
b = TextBlob("Simple is better than complex")
b.sentiment

Sentiment(polarity=0.06666666666666667, subjectivity=0.41904761904761906)

In [240]:
#pruebita['text'] = pruebita['text'].apply(lambda x: spell_correcter(x))
#pruebita.text = pruebita.text.apply(lambda txt: ''.join(TextBlob(txt).correct()))

pruebita['analisis_sentimientos'] = pruebita['text'].apply(lambda x: (TextBlob(str(x)).sentiment))
pruebita.head()

,keyword,location,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones,isRealPlace,longitud_tweet,letras_seguidas,text_2,analisis_sentimientos
1,null,null,"[forest, fire, near, la, range, ask, canada]",1,0,0,1,False,40,1,forest fire near la range ask canada,"(0.1, 0.4)"
2,null,null,"[all, residents, asked, to, shelter, in, place, are, being, notified, by, officers, no, other, evacuation, or, shelter, in, place, orders, are, expected]",1,0,0,1,False,130,2,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,"(-0.018750000000000003, 0.3875)"
3,null,null,"[, people, receive, wildfires, evacuation, orders, in, california, ]",1,0,0,1,False,60,1,people receive wildfires evacuation orders in california,"(0.0, 0.0)"
4,null,null,"[just, got, sent, this, photo, from, ruby, alaska, as, smoke, from, wildfires, hours, into, a, school, ]",1,0,0,1,False,90,2,just got sent this photo from ruby alaska as smoke from wildfires hours into a school,"(0.0, 0.0)"
5,null,null,"[rockyfire, update, , california, why, , closed, in, both, directions, due, to, lake, county, fire, , fire, wildfires]",1,0,0,1,False,110,1,rockyfire update california why closed in both directions due to lake county fire fire wildfires,"(-0.1125, 0.2375)"


In [241]:
train_limpio['sentimiento'] = train_limpio['text'].apply(lambda x: (TextBlob(str(x)).sentiment)[0])
train_limpio['objetividad'] = train_limpio['text'].apply(lambda x: (TextBlob(str(x)).sentiment)[1])

In [242]:
test_limpio['sentimiento'] = test_limpio['text'].apply(lambda x: (TextBlob(str(x)).sentiment)[0])
test_limpio['objetividad'] = test_limpio['text'].apply(lambda x: (TextBlob(str(x)).sentiment)[1])

In [243]:
train_limpio.head()

,keyword,location,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones,isRealPlace,longitud_tweet,letras_seguidas,sentimiento,objetividad
0,null,null,our deeds are the reason of this earthquake may allah forgive us all,1,0,0,1,False,70,2,0.00000,0.0000
1,null,null,forest fire near la ronge sask canada,1,0,0,1,False,40,1,0.10000,0.4000
2,null,null,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,0,1,False,130,2,-0.01875,0.3875
3,null,null,people receive wildfires evacuation orders in california,1,0,0,1,False,60,1,0.00000,0.0000
4,null,null,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0,0,1,False,90,2,0.00000,0.0000


# Stemming

In [244]:
#Prueba basica
pruebita = train_limpio.iloc[1:150].copy()

pruebita['stemmed_text'] = pruebita['text'].apply(lambda x: ' '.join([stemmer.stem(y) for y in x.split(' ')]))
pruebita.head()

,keyword,location,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones,isRealPlace,longitud_tweet,letras_seguidas,sentimiento,objetividad,stemmed_text
1,null,null,forest fire near la ronge sask canada,1,0,0,1,False,40,1,0.10000,0.4000,forest fire near la rong sask canada
2,null,null,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,0,1,False,130,2,-0.01875,0.3875,all resid ask to shelter in place are be notifi by offic no other evacu or shelter in place order are expect
3,null,null,people receive wildfires evacuation orders in california,1,0,0,1,False,60,1,0.00000,0.0000,peopl receiv wildfir evacu order in california
4,null,null,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0,0,1,False,90,2,0.00000,0.0000,just got sent this photo from rubi alaska as smoke from wildfir pour into a school
5,null,null,rockyfire update california hwy closed in both directions due to lake county fire cafire wildfires,1,0,0,1,False,110,1,-0.11250,0.2375,rockyfir updat california hwi close in both direct due to lake counti fire cafir wildfir


In [245]:
# Dio bien, sigo con el resto

In [246]:
train_limpio['text'] = train_limpio['text'].apply(lambda x: ' '.join([stemmer.stem(y) for y in x.split(' ')]))

In [247]:
test_limpio['text'] = test_limpio['text'].apply(lambda x: ' '.join([stemmer.stem(y) for y in x.split(' ')]))

In [248]:
test_limpio.head(1)

,id,keyword,location,text,cantidad_links,cantidad_signos_pregunta,repeticiones,isRealPlace,longitud_tweet,letras_seguidas,sentimiento,objetividad
0,0,null,null,just happen a terribl car crash,0,0,1,False,40,2,-1.0,1.0


In [249]:
test_limpio.dtypes

id                            int64
keyword                      object
location                     object
text                         object
cantidad_links                int64
cantidad_signos_pregunta      int64
repeticiones                  int64
isRealPlace                    bool
longitud_tweet                int64
letras_seguidas               int64
sentimiento                 float64
objetividad                 float64
dtype: object

## Caso abarcativo: agrupo por palabras y me quedo con las mas repetidas

In [250]:
import re 
from string import punctuation

palabras_por_separado = train_limpio.copy()
palabras_por_separado.set_index(['target'],inplace=True)
palabras_por_separado = palabras_por_separado['text'].str.split(expand=True).stack().to_frame()
palabras_por_separado.columns = ['word']
#palabras_por_separado['word'] = palabras_por_separado['word'].str.replace(rf'[{punctuation}]', '')
palabras_por_separado.reset_index(inplace=True)
del palabras_por_separado['level_1']
palabras_mas_comunes = palabras_por_separado.groupby('word')['target'].count().rename('cantidad_apariciones').reset_index().nlargest(400,'cantidad_apariciones')
palabras_mas_comunes.head()

,word,cantidad_apariciones
12554,the,3168
0,a,2090
6527,in,1904
12797,to,1903
9149,of,1759


In [251]:
for i in palabras_mas_comunes.word.values:
    train_limpio[i]= train_limpio['text'].apply(lambda x: x.count(i))
    test_limpio[i]= test_limpio['text'].apply(lambda x: x.count(i))

In [252]:
test_limpio.head(2)

,id,keyword,location,text,cantidad_links,cantidad_signos_pregunta,repeticiones,isRealPlace,longitud_tweet,letras_seguidas,...,sinkhol,spill,survivor,,big,harm,heart,kid,part,phone
0,0,null,null,just happen a terribl car crash,0,0,1,False,40,2,...,0,0,0,0,0,0,0,0,0,0
1,2,null,null,heard about earthquak is differ citi stay safe everyon,0,0,1,False,70,2,...,0,0,0,0,0,0,0,0,0,0


## Similar al anterior: agrupo por 2-shingles y me quedo con los que mas veces aparecen 
### En shingles puede apreciarse tambien informacion valiosa

In [253]:
palabras_por_separado = train_limpio.copy()
palabras_por_separado.set_index(['target'],inplace=True)
palabras_por_separado = palabras_por_separado['text'].str.split(expand=True).stack().to_frame()
palabras_por_separado.columns = ['word']
palabras_por_separado = palabras_por_separado.reset_index()
palabras_por_separado = palabras_por_separado[['word']]
palabras_por_separado.head()

,word
0,our
1,deed
2,are
3,the
4,reason


In [254]:
palabras_shifteadas = train_limpio.copy()
palabras_shifteadas.set_index(['target'],inplace=True)
palabras_shifteadas = palabras_shifteadas['text'].str.split(expand=True).stack().to_frame()
palabras_shifteadas.columns = ['next_word']
palabras_shifteadas = palabras_shifteadas.reset_index()
palabras_shifteadas = palabras_shifteadas[['next_word']]
palabras_shifteadas = palabras_shifteadas.shift(-1)
palabras_shifteadas.head()

,next_word
0,deed
1,are
2,the
3,reason
4,of


In [255]:
palabras_shifteadas.shape

(101060, 1)

In [256]:
dos_shingles = palabras_por_separado.merge(palabras_shifteadas, left_index=True, right_index=True)
dos_shingles.head()

,word,next_word
0,our,deed
1,deed,are
2,are,the
3,the,reason
4,reason,of


In [257]:
dos_shingles['shingle'] = dos_shingles['word']+' '+dos_shingles['next_word']
dos_shingles['valor'] = 1
dos_shingles.head()

,word,next_word,shingle,valor
0,our,deed,our deed,1
1,deed,are,deed are,1
2,are,the,are the,1
3,the,reason,the reason,1
4,reason,of,reason of,1


In [258]:
dos_shingles_agrupados = dos_shingles.groupby('shingle').valor.sum().rename('cantidad_apariciones').to_frame()
shingles_mas_comunes = dos_shingles_agrupados.nlargest(400,'cantidad_apariciones')
shingles_mas_comunes.head()

,cantidad_apariciones
shingle,
in the,302
of the,247
like a,130
on the,129
to the,121


In [259]:
for i in shingles_mas_comunes.index:
    train_limpio[i]= train_limpio['text'].apply(lambda x: x.count(i))
    test_limpio[i]= test_limpio['text'].apply(lambda x: x.count(i))

## Columna si contiene insultos, o la cantidad que tiene

In [260]:
#Estoy usando el train original porque para este prueba simple uso los textos aun no separados por palabra
insults_list = ['idiot','fuck','moron','scum','bitch','shit','bastard','cunt','ridicul','tramp']
mean_insult = [train[train['text'].str.contains(i)].target.mean() for i in insults_list]
times_insult_used = [len(train[train['text'].str.contains(i)]) for i in insults_list]

In [261]:
print(insults_list)
print(times_insult_used)
print(mean_insult)

['idiot', 'fuck', 'moron', 'scum', 'bitch', 'shit', 'bastard', 'cunt', 'ridicul', 'tramp']
[4, 67, 1, 1, 10, 57, 1, 4, 4, 1]
[0.5, 0.208955223880597, 0.0, 0.0, 0.0, 0.3157894736842105, 0.0, 0.0, 0.25, 1.0]


In [262]:
def insults(text):
    insults = ['idiot','fuck','moron','scum','bitch','shit','bastard','cunt','ridicul','tramp']
    amount = 0
    for word in text.split(' '):
        if(word in insults):
            amount = amount +1
    return(amount)

In [263]:
#train_limpio['insultos_por_tweet'] = train_limpio['text'].apply(lambda x: insults(x) )
test_limpio['insultos_por_tweet'] = test_limpio['text'].apply(lambda x: insults(x) )

In [264]:
#No deberia ser necesario, pero tengo que convertir esto a string, aunque ya lo sea para que ande
train_limpio['insultos_por_tweet'] = train_limpio['text'].apply(lambda x: insults(x) )

In [265]:
train_limpio[train_limpio['insultos_por_tweet']==1].target.mean()

0.20689655172413793

In [266]:
# Si bien algunos aparecen mucho menos que otros, en todos se cumple que estan por debajo del 0.5

In [267]:
# Remuevo estas palabras de los textos
# Al final no las remuevo, por que lo haria?
#train_limpio['text']=train_limpio['text'].apply(lambda x: [item for item in x if item not in insults_list])

## Remover expresiones

In [268]:
#Estoy usando el train original porque para este prueba simple uso los textos aun no separados por palabra
expressions = ['lol','omg','wtf','lmao','imo']  #agregar mas,no importa que el train no tenga, si el test si,joya
mean_expressions = [train[train['text'].str.contains(i)].target.mean() for i in expressions]
times_expressions_used = [len(train[train['text'].str.contains(i)]) for i in expressions]

In [269]:
print(expressions)
print(times_expressions_used)
print(mean_expressions)

['lol', 'omg', 'wtf', 'lmao', 'imo']
[54, 6, 8, 14, 15]
[0.12962962962962962, 0.3333333333333333, 0.375, 0.0, 0.3333333333333333]


In [270]:
def expressions_counter(text):
    expressions = ['lol','omg','wtf','lmao','imo']
    amount = 0
    for word in text:
        if(word in expressions):
            amount = amount +1
    return(amount)

In [271]:
train_limpio['expresiones_por_tweet'] = train_limpio['text'].apply(lambda x: expressions_counter(x))
test_limpio['expresiones_por_tweet'] = test_limpio['text'].apply(lambda x: expressions_counter(x) )

In [272]:
# Remuevo estas palabras de los textos
# Al final no las remuevo, por que lo haria?
#train_limpio['text']=train_limpio['text'].apply(lambda x: [item for item in x if item not in expressions])

## Cuento pronombres personales,esto se habia hecho en el TP1 y dio buenos resultados

In [273]:
#Estoy usando el train original porque para este prueba simple uso los textos aun no separados por palabra
personal_pronouns = ['i','me','you','she','her','him','he','his','we','us','them','they','im']  #agregar mas,no importa que el train no tenga, si el test si,joya
mean_personal_pronouns = [train[train['text'].str.contains(i)].target.mean() for i in personal_pronouns]
times_personal_pronoun_used = [len(train[train['text'].str.contains(i)]) for i in personal_pronouns]

In [274]:
print(personal_pronouns)
print(mean_personal_pronouns)
print(times_personal_pronoun_used)

['i', 'me', 'you', 'she', 'her', 'him', 'he', 'his', 'we', 'us', 'them', 'they', 'im']
[0.4389975213439824, 0.43219954648526077, 0.2056921086675291, 0.4017857142857143, 0.42958579881656805, 0.7189189189189189, 0.4109311740890688, 0.37099494097807756, 0.4495515695067265, 0.4447797223898612, 0.3557692307692308, 0.3006134969325153, 0.5167630057803468]
[7262, 2205, 773, 224, 845, 185, 3952, 593, 892, 1657, 104, 163, 865]


In [275]:
for i in personal_pronouns:
    train_limpio[i]= train_limpio['text'].apply(lambda x: x.split(' ').count(i))
    test_limpio[i]= test_limpio['text'].apply(lambda x: x.split(' ').count(i))

## Palabras mas comunes (esta parte no se si deberia quedar así basandome en las del TP1 o rehacer eso ahora que está mas limpio el train o hacerlo de otra forma)

In [276]:
palabras = ['hiroshima','bombing','kill','california','suicide','fire','forest','train','accident','disaster'\
                ,'new','crash','storm','police','nuclear','two','dead','died','years']
for i in palabras:
    train_limpio[i]= train_limpio['text'].apply(lambda x: x.split(' ').count(i))
    test_limpio[i]= test_limpio['text'].apply(lambda x: x.split(' ').count(i))

In [277]:
#muchas de estas palabras ya estaban agregadas por otros medios

## Guardo los csvs limpios para mas adelante

In [278]:
train_limpio.to_csv('train_limpio_con_stemming.csv', index=False)

In [279]:
test_limpio.to_csv('test_limpio_con_stemming.csv', index=False)

## Vectorizo los textos (quiza la parte mas importante de todo el cleaning y apenas lo toque)

In [280]:
train_solo_texto = train_limpio[['id','text']].copy()

KeyError: "['id'] not in index"

In [ ]:
train_solo_texto['text'] = train_solo_texto['text'].apply(lambda x: ' '.join(x))

In [ ]:
train_solo_texto.head()

In [ ]:
train_solo_texto.groupby('text')['id'].transform('count')

In [ ]:
train_solo_texto.groupby('text').id.count().to_frame().reset_index().nlargest(40,'id')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#agregarle n-shingles despues
cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(train_solo_texto.text)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = train_solo_texto.index
data_dtm

In [ ]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
stemmed = [porter.stem(word) for word in tokens]
#train_solo_texto['stemmed'] = train_solo_texto['text'].apply(lambda x: porter.stem(word) for word in x)

In [ ]:
train_solo_texto['stemmed'] = train_solo_texto['text'].apply(lambda x: porter.stem(word) for word in x)